# Aufgabe 1 (5P)

## a)
Nutzen Sie vom Datensatz „pvtest.csv“ die Werte 'Edaily','Dci','Dcp','Dcu','Temp1','hour‘ und erstellen Sie einen DataFrame
als Zeitreihe, ähnlich wie unten abgebildet. Auf Grundlage dieser Daten soll der Energieertrag Dcp immer in 5 Minuten in
der Zukunft prognostiziert werden. (0.5P)

<img src='images/1a.png' width=400>

In [88]:
import pandas as pd
from tensorflow.tools.docs.doc_controls import header

df_1 = pd.read_csv('pvtest.csv')
df_1['Time'] = pd.to_datetime(df_1['Time'], format='%d.%m.%y %H:%M')
df_1['hour'] = df_1['Time'].dt.hour
df_1 = df_1.set_index('Time')
columns = ['Edaily','Dci','Dcp','Dcu','Temp1','hour']
df_1 = df_1[columns]
df_1.tail(15)

,Edaily,Dci,Dcp,Dcu,Temp1,hour
Time,,,,,,
2021-01-26 16:25:00,3.108,0.192504,113,587,14.0,16
2021-01-26 16:30:00,3.122,0.111913,62,554,14.0,16
2021-01-26 16:35:00,3.130,0.053846,28,520,14.0,16
2021-01-26 16:40:00,3.137,0.041176,21,510,14.0,16
2021-01-26 16:45:00,3.141,0.024283,11,453,14.0,16
2021-01-26 16:50:00,3.144,0.000000,0,271,14.0,16
2021-01-26 16:55:00,3.145,0.000000,0,257,13.0,16
2021-01-26 17:00:00,3.145,0.000000,0,251,12.0,17
2021-01-26 17:05:00,3.145,0.000000,0,250,12.0,17


## b)
Teilen Sie die Daten sinnvoll in Lern-und Testdatensatz auf, so dass die zeitlichen Reihenfolgen bestehen bleiben ! (0.5P)

In [105]:
from sklearn.model_selection import train_test_split

x = df_1.drop('Dcp', axis=1)
y = df_1[['Dcp']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

## c)
Standardisieren Sie die Daten mit dem Standardscaler und restrukturieren Sie die Zeitreihe in einzelne Datenfenster, so dass
Sie aus allen vorhandenen 6 Features aus den letzten drei Stunden immer jeweils den Energieertrag für die nächsten 5
Minuten vorhersagen können. (1,5P)

In [90]:
from sklearn.preprocessing import StandardScaler

# standardize training data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# d)
Definieren und kompilieren Sie jetzt ein passendes Modell in Keras, welches mind. eine LSTM-Layer nutzt und DropoutLayer einfügt, um „Überanpassung“ zu verhindern. (1P)

## e)
Trainieren Sie das Modell mit allen Lerndaten über mind. 100 Epochen, nutzen Sie ggf. auch callbacks, um den Lernprozess
zu verkürzen. Verbessern Sie Ihr Training und Ihre Modellarchitektur so lange, bis Sie einen Mean absolute Error von mind.
50 erzielen! (1P)

# f)
Visualisieren Sie Ihre Prognoseergebnisse, so ähnlich wie unten angegeben im Vergleich gemessener Daten mit
prognostizierten Werten! (0.5P)

<img src="images/1f.png" width=900>